In [40]:
import AdaNet_CIFAR_10_feature_extraction as AdaFE
import time
import importlib
import math
import random

In [2]:
importlib.reload(AdaFE)

<module 'AdaNet_CIFAR_10_feature_extraction' from 'C:\\Users\\David\\Documents\\דוקטורט\\קורס יישום אלגוריתמים לומדים 372.2.5214\\ex4\\adanet\\AdaNet_CIFAR_10_feature_extraction.py'>

In [3]:
cat_dog = AdaFE.CF10_pairs('cat','dog')

In [4]:
start_time = time.time()
X_train, y_train, X_test, y_test = AdaFE.train_test_dataset(cat_dog)
print(time.time() - start_time)

0.12780475616455078


In [2]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.python.framework import ops
import os.path
import csv

In [6]:
ops.reset_default_graph()

# Create graph
sess = tf.Session()

In [7]:
# set for reproducible results
seed = 99
np.random.seed(seed)
tf.set_random_seed(seed)

In [8]:
x_vals_train = X_train
x_vals_test = X_test
y_vals_train = y_train
y_vals_test = y_test

In [234]:
cfg = {}
cfg['maxNodes'] = np.dot(np.array(50),np.ones((1,5)))
cfg['maxWeightNorms'] = np.dot(np.array(10),np.ones(np.size(cfg['maxNodes']))) #capital lambda
cfg['pnorm'] = 2
cfg['maxWeightMag'] = np.dot(np.array(10),np.ones(np.size(cfg['maxNodes'])))
cfg['maxBiasMag'] = 10
cfg['complexityRegWeight'] = np.logspace(-4,-3,cfg['maxNodes'].shape[1])
cfg['normRegWeight'] = np.dot(np.array(0.1),np.ones(np.size(cfg['maxNodes']))) #beta
cfg['augment'] = True
cfg['augmentLayers'] = True
cfg['activationFunction'] = 'tanh'
cfg['numEpochs'] = 300
cfg['big_lambda'] = 1.01
cfg['featureMap'] = []

cfg['lossFunction'] = 'binary'
cfg['surrogateLoss'] = 'logistic'
cfg['javier'] = True

In [235]:
xs = twospirals(44100,540*1,0,1)
ys = 2*(xs[:,-1])-1
xs = xs[:,:-1] # python starts at 0, by slicing the end argument is not accounted MATLAB 1:end-1 == Python :-1

In [246]:
adaParams,history = adanet(xs, ys, cfg)

In [126]:
def twospirals(*args):  #N, degrees, start, noise
# Generate "two spirals" dataset with N instances.
# degrees controls the length of the spirals
# start determines how far from the origin the spirals start, in degrees
# noise displaces the instances from the spiral. 
#  0 is no noise, at 1 the spirals will start overlapping

    if len(args) < 1:
        N = 2000
    else:
        N = args[0]
    if len(args) < 2:
        degrees = 570
    else:
        degrees = args[1]
    if len(args) < 3:
        start = 90
    else:
        start = args[2]
    if len(args) < 5:
        noise = 0.2
    else:
        noise = args[3]
    
    deg2rad = (2*math.pi)/360
    start = start * deg2rad

    N1 = math.floor(N/2)
    N2 = N-N1
    
    n = start + np.dot(np.dot(np.sqrt(np.random.random((N1,1))),degrees),deg2rad)
    d1 = np.hstack([-np.cos(n)*n + np.dot(np.random.random((N1,1)),noise),np.sin(n)*n+np.dot(np.random.random((N1,1)),noise),np.zeros((N1,1))])
    
    n = start + np.dot(np.dot(np.sqrt(np.random.random((N1,1))),degrees),deg2rad)   
    d2 = np.hstack([np.cos(n)*n + np.dot(np.random.random((N2,1)),noise),-np.sin(n)*n+np.dot(np.random.random((N2,1)),noise),np.ones((N2,1),float)])
    
    data = np.vstack((d1,d2))
    return data

In [81]:
N=44100
degrees=540
start=0
noise=1
N1 = math.floor(N/2)
N2 = N-N1
deg2rad = (2*math.pi)/360
degrees = 570
noise = 0.2

In [253]:
#%% Activation function
def activation(h,afunc):
    choices = {'relu': np.maximum(0,h), 'tanh': np.tanh(h), 'sigmoid': 1./(1+np.exp(-h))}
    a = choices.get(afunc, h)
    #        error('activation function not supported')
    return a

In [159]:
# gradient of surrogate loss function
def slgrad(x, func):
    choices = {'logistic': np.exp(x)/(1+np.exp(x)), 'exp': np.exp(x)}
    valG = choices.get(func, x)
    return valG

In [160]:
# surrogate loss function
def slfunc(x, func):
    choices = {'logistic': np.log(1+np.exp(x)), 'exp': np.exp(x)}
    val = choices.get(func, x)
    return val

In [ ]:
%% Loss Function
function [loss,grad] = loss_function(w_k, h_k, reg_k, y, loss_notk,reg_notk, surrloss,lossfunc)
m = size(y,1);
switch lossfunc
    case 'binary'
        farg = 1 - loss_notk - w_k*h_k.*y;
        loss = 1/m * sum(slfunc(farg,surrloss)) + reg_notk + reg_k*abs(w_k);
        grad = -1/m * sum(slgrad(farg,surrloss).*y.*h_k) + sign(w_k)*reg_k;
%     case 'MSE'
%         farg = loss_notk + abs(w_k*h_k - y).^2;
%         loss = 1/m * sum(farg) + reg_notk + reg_k*abs(w_k).^2;
%         grad = ( -1/m * sum(farg.*h_k) +sign(w_k)*reg_k);
end
end

In [161]:
# Loss Function
def loss_function(w_k, h_k, reg_k, y, loss_notk,reg_notk, surrloss,lossfunc):
    m = np.size(y)
    if lossfunc == 'binary':
        farg = 1 - loss_notk - np.dot(w_k,h_k)*y
        loss = 1/m * slfunc(farg,surrloss).sum(axis=0) + reg_notk + np.dot(reg_k,np.abs(w_k))
        grad = -1/m * (slgrad(farg,surrloss)*y*h_k).sum(axis=0) + np.dot(np.sign(w_k),reg_k)
#     case 'MSE'
#         farg = loss_notk + abs(w_k*h_k - y).^2;
#         loss = 1/m * sum(farg) + reg_notk + reg_k*abs(w_k).^2;
#         grad = ( -1/m * sum(farg.*h_k) +sign(w_k)*reg_k);
    return loss,grad

In [279]:
# Initialization function
# implementation of Init method, Figure 5,  pg 15
def adanet_init(numExamples,numInputNodes,cfg):

# Initialize output weights (w)
    maxNodes = cfg['maxNodes']  # == shape=(1,5) [50,50,50,50,50] 
    maxLayers = max(cfg['maxNodes'].shape)  #length(cfg.maxNodes); ==5
    W = {} #W = cell(maxLayers,1);
    for k in range(maxLayers):  # for k = 1:maxLayers
        W[k] = np.zeros((np.int(maxNodes[0,k]),1))   #  W{k} = zeros(maxNodes(k),1);

# Assign current number of nodes and layers
    numLayers = 0    # number of hidden layers (no input or output layer considered)
    numNodes = np.zeros((max(maxNodes.shape),1)) # == shape=(5,1) zeros(length(maxNodes),1); #number of nodes in the hidden layers

# Initialize feed-in weights (u)
    u = {} # u = cell(1,1);
    #???u[0] = np.zeros((numInputNodes,1))       #% u{1} = zeros(numInputNodes,1);

#% Create initial distribution (uniform)
    D = np.dot(1/numExamples, np.ones((numExamples,1))) # D = 1/numExamples * ones(numExamples,1);

    adaInit = {}

    adaInit['W'] = W
    adaInit['W_bias'] = 0
    adaInit['u'] = u
    adaInit['D'] = D
    adaInit['S'] = numExamples
    adaInit['errLoss'] = W  # %same shape as W (maybe??)
    adaInit['numLayers'] = numLayers
    adaInit['maxLayers']= maxLayers
    adaInit['numNodes'] = numNodes
    adaInit['maxNodes'] = maxNodes
    #%cfg.pnorm = max(1.001,cfg.pnorm);
    adaInit['qnorm'] = 1/(1-1/cfg['pnorm']) #adaInit.qnorm = 1/(1-1/cfg.pnorm);
    adaInit['complexityRegWeight'] = cfg['complexityRegWeight'] #%lambda
    adaInit['normRegWeight'] = cfg['normRegWeight'] # %beta
    adaInit['surrogateLoss'] = cfg['surrogateLoss']
    adaInit['lossFunction'] = cfg['lossFunction']
    adaInit['maxWeightMag'] = cfg['maxWeightMag']
    adaInit['maxBiasMag'] = cfg['maxBiasMag']
    adaInit['activationFunction'] = cfg['activationFunction']
    adaInit['augment'] = cfg['augment']
    adaInit['augmentLayers'] = cfg['augmentLayers']
    adaInit['featureMap'] = cfg['featureMap']
    return adaInit

In [162]:
maxNodes = cfg['maxNodes']
maxLayers = cfg['maxNodes'].shape[1]

In [ ]:

    W = cell(maxLayers,1);
    for k in range(maxLayers):
        W{k} = zeros(maxNodes(k),1)

In [200]:
W = [np.empty([maxLayers,1])]


In [201]:
W[0] = np.zeros((np.int(maxNodes[0,0]),1))

In [206]:
W = [np.empty([maxLayers,1])]
for k in range(maxLayers):
    print(k)
    W[k] = np.zeros((np.int(maxNodes[0,k]),1))

0
1


IndexError: list assignment index out of range

In [245]:
#%% Main function for adanet
def adanet(Xdata,ydata, cfg):
    history = {}
    history['newL'] = [];
    history['numL'] = [];
    history['newN'] = [];
    history['numN'] = [];
    history['jk_best'] = [];
    history['Wt'] = {};
    history['ut'] = {};
    history['W_bias'] = [];
    history['activationFunction'] = cfg['activationFunction']

#%Xdata = [Xdata, Xdata.^2, prod(Xdata,2)];
#if ~isempty(cfg.featureMap);
#    Xdata = cfg.featureMap(Xdata);
    if len(cfg['featureMap'])!=0:
        Xdata = cfg['featureMap'][Xdata]
    if cfg['augment']:
        Xdata = np.hstack((np.ones((Xdata.shape[0],1)),Xdata))      #  Xdata = [ones(size(Xdata,1),1),Xdata];
    if cfg['augment'] and cfg['augmentLayers']:
        Xdata = Xdata[:,1:] #Xdata = Xdata(:,2:end); #%revert Xdata! We will account for it in NewNodes

    numExamples = Xdata.shape[0] #size(Xdata,1); #% number of training examples (m)
    numInputNodes = Xdata.shape[1] #size(Xdata,2);

    if ydata.shape[0] != numExamples:    #size(ydata,1) ~= numExamples
       raise Exception('dim check', 'Xdata and ydata must have same number of examples (i.e. rows)')

    adaParams = adanet_init(numExamples,numInputNodes,cfg) #%see below

    adaParams['lossStore'] = []
    adaParams['numInputNodes'] = numInputNodes
    adaParams['numOutputNodes'] = 1 #ydata.shape[1] has only 1dim, in cases of more labels will have relevant value #size(ydata,2);
    T = cfg['numEpochs'] #% num rounds from paper

    xloss_end = round(numExamples/2)
    step=0
    for t in range(T):
        idcs_t = np.random.permutation(range(numExamples)) #idcs_t = randperm(numExamples); #% shuffle samples uniformly every epoch
    #% split data into two mini batches, one for loss, one for finding new
    #% nodes (Appendix E, page 17)
    #%     Xloss = Xdata(1:xloss_end,:);
    #%     Xnewnodes = Xdata(xloss_end+1:end,:);
        lossIdcs = idcs_t[:xloss_end] #lossIdcs = idcs_t(1:xloss_end);
        newIdcs = idcs_t[xloss_end:] #newIdcs = idcs_t(xloss_end:end);
    #% N = sum(adaParams.numNodes);
    
    #% Compute Forward Pass + Complexity Measures
        numLayers = adaParams['numLayers']
    
        H = {} #H = cell(numLayers,1);  % hidden unit values per layer
        Rloss = np.zeros((numLayers+1,1)) #Rloss = zeros(numLayers+1,1); % complexity measures per layer
        Rnew = Rloss.copy()
        W = adaParams['W']
        u = adaParams['u']
        for k in range(numLayers):
            if k == 0:  # k==1
                Hk = Xdata
                actFunc = 'none'
            else:
                Hk = H[k-1]   #H{k-1}
                actFunc = adaParams['activationFunction']

            if adaParams['augmentLayers']:
                Hk = np.hstack((np.ones((Hk.shape[0],1)),Hk))   # Hk = [ones(size(Hk,1),1),Hk];
                
            H[k] = np.dot(activation(Hk,actFunc),u[k])  #H{k} = activation(Hk,actFunc)*u{k};
            #% Calculate loss function
            if cfg['lossFunction'] == 'binary':
                Rloss(k) = RademacherComplexity(H{k}(lossIdcs,:));  #Rloss(k) = RademacherComplexity(H{k}(lossIdcs,:));
                Rnew(k) = RademacherComplexity(H{k}(newIdcs,:));  #Rnew(k) = RademacherComplexity(H{k}(newIdcs,:));
   # %             case 'MSE'
   # %                 Rloss(k) = GaussianComplexity(H{k}(lossIdcs,:)); %equivalent with gaussian, instead of binary, noise
   # %                 Rnew(k) = GaussianComplexity(H{k}(newIdcs,:));
            else:
                error('loss not supported')

            Rloss(k) = reg(Rloss(k),adaParams.complexityRegWeight(k),adaParams.normRegWeight(k));
            Rnew(k) = reg(Rnew(k),adaParams.complexityRegWeight(k),adaParams.normRegWeight(k));
        end

        % score the existing nodes in the network
        [d_kj, d_bias] = ExistingNodes;  % pg 7, line 3
        [dn_k, un_k] = NewNodes; %pg 7, line 4
        [jk_best, e_t, numLayers, numNodes] = BestNode; %pg 7, line 5
        adaParams.numLayers = numLayers;
        adaParams.numNodes = numNodes;
        adaParams.u = u;
        % Move forward a step and update distributions
        Wt = ApplyStep(jk_best,adaParams.surrogateLoss);
        [Dt,St] = updateDistribution(Wt);
        % Pack things back into adaParams
        adaParams.D = Dt;
        adaParams.S = St;
        adaParams.W = Wt;
        adaParams.errLoss = e_t;

        % Store history for debugging only
        history.Wt{end+1} = adaParams.W;
        history.ut{end+1} = adaParams.u;
        history.numL(end+1) = numLayers;
        history.numN(end+1,:) = adaParams.numNodes;

        fComplete = 0;
        for k = 1:numLayers
            for j = 1:numNodes(k)
                fComplete = fComplete + W{k}(j)*H{k}(:,j);
            end
        end
        fComplete = fComplete + adaParams.W_bias;
        switch cfg.lossFunction
            case 'binary'
                adaParams.lossStore(end+1) = mean(slfunc(1-ydata.*fComplete, adaParams.surrogateLoss));
    %         case 'MSE'
    %             adaParams.lossStore(end+1) =mean(abs(ydata - fComplete).^2);
        end
        plotEpochs = 5;
        if mod(step,plotEpochs)==0
            figure(2), plot(adaParams.lossStore)
            titleStr = 'numNodes = (';
            for ni = 1:length(numNodes)-1,
                titleStr = sprintf([titleStr,'%d,'],numNodes(ni));
            end
            titleStr = sprintf([titleStr,'%d)'],numNodes(end));
            title(titleStr)
            pause(.1)
        end
        step = step+1;



    end
    return adaParams,history

In [330]:
xloss_end = round(numExamples/2)
step=0

idcs_t = np.random.permutation(range(numExamples)) #idcs_t = randperm(numExamples); #% shuffle samples uniformly every epoch
#% split data into two mini batches, one for loss, one for finding new
#% nodes (Appendix E, page 17)
#%     Xloss = Xdata(1:xloss_end,:);
#%     Xnewnodes = Xdata(xloss_end+1:end,:);
lossIdcs = idcs_t[:xloss_end] #lossIdcs = idcs_t(1:xloss_end);
newIdcs = idcs_t[xloss_end:] #newIdcs = idcs_t(xloss_end:end);

In [527]:
#%% Complexity functions
def RademacherComplexity(H):
    M = H.shape[0] #M = size(H,1);
    radNoise = np.random.standard_normal((M,1))  #radNoise = randn(M,1);
    radNoise[radNoise <= 0] = -1
    radNoise[radNoise > 0] = 1

    R = 1/M*np.dot(radNoise.conj().transpose(),H.sum(axis=1))  #R = 1/M*radNoise'*sum(H,2);

    return R

In [249]:
    if cfg['augment']:
        Xdata = np.hstack((np.ones((Xdata.shape[0],1)),Xdata))      #  Xdata = [ones(size(Xdata,1),1),Xdata];
    if cfg['augment'] and cfg['augmentLayers']:
        Xdata = Xdata[:,1:]

array([2, 8, 7, 5, 6, 4, 0, 1, 9, 3])

In [519]:
radNoise = np.random.standard_normal((3,1))

In [522]:
radNoise[radNoise <= 0] = -1

array([[-0.74012907],
       [-0.14266716],
       [ 0.25564079]])

In [523]:
radNoise[radNoise <= 0] = -1

In [526]:
radNoise

array([[-1.],
       [-1.],
       [ 1.]])

In [525]:
R = 1/3*np.dot(radNoise.conj().transpose(),H.sum(axis=1))

In [560]:
test = {}

In [566]:
if test.get(0,1) == 1:
    test[0] = -1
else:
    test[max(test.keys()) + 1] = 'holla'

In [567]:
test

{0: -1, 1: 'holla', 2: 'holla'}

In [379]:
Xdata[0]

array([0.74918961, 1.92843202])

In [380]:
numExamples = Xdata.shape[0] #size(Xdata,1); #% number of training examples (m)
numInputNodes = Xdata.shape[1]

In [381]:
numExamples

44100

In [424]:
np.zeros((numLayers+1,1))

array([[0.]])

In [383]:
numLayers = 0

In [269]:
Rloss = np.zeros((numLayers+1,1))

In [271]:
Rloss.shape

(1, 1)

In [425]:
Rnew = Rloss.copy()

In [272]:
Hk = Xdata
actFunc = 'none'

In [273]:
Hk = np.hstack((np.ones((Hk.shape[0],1)),Hk))

In [275]:
Hk[0]

array([1.        , 0.74918961, 1.92843202])

In [277]:
H = {}

In [282]:
u = {} # u = cell(1,1);
u[0] = np.zeros((numInputNodes,1))  
k=0

In [283]:
H[k] = np.dot(activation(Hk,actFunc),u[k])

ValueError: shapes (44100,3) and (2,1) not aligned: 3 (dim 1) != 2 (dim 0)

In [287]:
activation(Hk,actFunc).shape

(44100, 3)

In [289]:
u[k]

array([[0.],
       [0.]])

In [314]:
adaParams.keys()

dict_keys(['W', 'W_bias', 'u', 'D', 'S', 'errLoss', 'numLayers', 'maxLayers', 'numNodes', 'maxNodes', 'qnorm', 'complexityRegWeight', 'normRegWeight', 'surrogateLoss', 'lossFunction', 'maxWeightMag', 'maxBiasMag', 'activationFunction', 'augment', 'augmentLayers', 'featureMap', 'lossStore', 'numInputNodes', 'numOutputNodes'])

In [316]:
adaParams['numNodes'].shape

(5, 1)

In [518]:
# New nodes function
def NewNodes():
    global errNew
    numNodes = adaParams['numNodes']
    maxNodes = adaParams['maxNodes']  # shape=(1,5) , [50,50,50,50,50]
    q = adaParams['qnorm']
    p = cfg['pnorm']
    idx = np.minimum(numLayers+1,adaParams['maxLayers'])
    errNew = {} # cell(idx,1);
    un_k = {} # cell(idx,1);
    dn_k = {} # cell(idx,1);

    Dnew = adaParams['D'][newIdcs,0] #adaParams.D(newIdcs);
    Dnew = Dnew/Dnew.sum(axis=0) # Dnew./sum(Dnew);
    m = max(newIdcs.shape) # length(newIdcs);
    S = adaParams['S']  # number of samples (44100)
    if cfg['javier']:
        # adding per J.Gonzalva recommendation
        big_lambdas = np.zeros((adaParams['maxLayers'],1),float)  # ==shape(5,1) big_lambdas = zeros(adaParams.maxLayers,1)  
        for n in range(-1,adaParams['maxLayers']-1):  #for n = 0:adaParams.maxLayers-1 => 0..4
            if n == -1:                              # if n == 0
                n_i=adaParams['numInputNodes']
            else:
                n_i = maxNodes[0,n]    # maxNodes(n)
                big_lambdas[n] = cfg['big_lambda']/n_i**(1/q)     # cfg.big_lambda/n_i^(1/q);

    for k in range (idx):   #for k = 1:idx                                       # pg 8, Fig 4, Line 1
        if not(cfg['javier']):   #if ~cfg.javier
            Ck = cfg['maxWeightMag'][k]  # shape=(5,) [10,10,10,10,10]   Ck = cfg.maxWeightMag(k); 
            Lambda_k = cfg['maxWeightNorms'][k]   # shape=(5,) [10,10,10,10,10] Lambda_k = cfg.maxWeightNorms(k);
        else:
            Ck = Rnew[k]/4**(k-1) #  Ck = Rnew(k)/4^(k-1); % per Javier's recommendation
            Lambda_k = big_lambdas[k]  #Lambda_k = big_lambdas(k);
         # % adding per J.Gonzalva recommendation

        if k==0:   #if k==1, 
            nodeNum_km1 = numInputNodes  #nodeNum_km1 = numInputNodes;
            Hk = Xdata  #Hk = Xdata;
            actFunc = 'none'
        else:
            nodeNum_km1 = numNodes[k-1]  #nodeNum_km1 = numNodes(k-1);
            Hk = H[k-1]  # Hk = H{k-1};
            actFunc = adaParams['activationFunction']

        if (numNodes[k] < maxNodes[k]) and (nodeNum_km1 > 0):  #if (numNodes(k) < maxNodes(k)) && (nodeNum_km1 > 0)   %pg 8, Fig 4, line 2
            if adaParams['augmentLayers']:
                Hk = np.hstack((np.ones((Hk.shape[0],1)),Hk))  #Hk = [ones(size(Hk,1),1),Hk];
   
            if cfg['lossFunction'] == 'binary': 
                M_kminus1 = np.dot(((ydata[newIdcs].reshape(-1,1)*activation(Hk[newIdcs],actFunc)).conj().T),Dnew) #M_kminus1 = bsxfun(@times,ydata(newIdcs,:),activation(Hk(newIdcs,:),actFunc))'*Dnew;
                M_qnorm = np.linalg.norm(M_kminus1,q)   #M_qnorm = norm(M_kminus1,q);
                errNew[k] = Ck/2*(1-Lambda_k/Ck * M_qnorm)  #errNew{k} = Ck/2*(1-Lambda_k/Ck * M_qnorm);  % pg 8, Fig 4, line 3
#%                     case 'MSE'
#%                         M_kminus1 =  (abs(bsxfun(@minus,ydata(newIdcs,:),activation(Hk(newIdcs,:),actFunc))).^2)'*Dnew;
#%                         M_qnorm = norm(M_kminus1,q);
#%                         errNew{k} = Ck/2*(1-Lambda_k/Ck * M_qnorm);  % pg 8, Fig 4, line 3


            if p == 1:
                _,bMax = (np.abs(M_kminus1)).max(),(np.abs(M_kminus1)).argmax() #[~,bMax] = max(abs(M_kminus1));
                M_normalized = np.zeros(M_kminus1.shape)  #M_normalized = zeros(size(M_kminus1));
                M_normalized[bMax] = 1/abs(M_kminus1[bMax])  #M_normalized(bMax) = 1/abs(M_kminus1(bMax));
                un_k[k] = Lambda_k * M_normalized*np.sign(M_kminus1)  #un_k{k} = Lambda_k * M_normalized.*sign(M_kminus1);
            else:
                un_k[k] =  Lambda_k * (np.abs(M_kminus1)**(q-1))*np.sign(M_kminus1)/(M_qnorm**(q/p)) # un_k{k} =  Lambda_k * (abs(M_kminus1).^(q-1)).*sign(M_kminus1)/(M_qnorm^(q/p) ); % line 4

        else:
            errNew[k] = 1/2*Ck   # errNew{k} = 1/2*Ck;  % line 5,


        if abs(errNew[k] - 1/2*Ck) <= Rnew[k]*m/(2*S):  #if abs(errNew{k} - 1/2*Ck) <= Rnew(k)*m/(2*S)   % line 6
            dn_k[k] = 0  # dn_k{k} = 0;                                               % line 7
        else:
            dn_k[k] = errNew[k] - Ck/2 - np.sign(errNew[k] - Ck/2)*Rnew[k]*m/(2*S) #dn_k{k} = errNew{k} - Ck/2 - sign(errNew{k} - Ck/2)*Rnew(k)*m/(2*S);  %line 8

    return dn_k, un_k

In [444]:
big_lambdas = np.zeros((adaParams['maxLayers'],1),float)

In [482]:
for k in range(1):
    Lambda_k = cfg['maxWeightNorms'][k]
    Ck = cfg['maxWeightMag'][k]

In [462]:
(ydata[newIdcs].reshape(-1,1)*activation(Hk[newIdcs],actFunc))

array([[-1.        ,  0.8900217 ,  4.71879179],
       [ 1.        , -0.30639893, -1.6339166 ],
       [ 1.        ,  3.88255545, -6.13775672],
       ...,
       [-1.        ,  1.71377761, -7.51738881],
       [-1.        , -4.85376002, -7.14712327],
       [ 1.        ,  1.0274849 ,  4.92539322]])

In [470]:
M_kminus1 = np.dot(((ydata[newIdcs].reshape(-1,1)*activation(Hk[newIdcs],actFunc)).conj().T),Dnew)

In [513]:
p = cfg['pnorm']

In [474]:
M_qnorm = np.linalg.norm(M_kminus1,q)

In [494]:
_,bMax = (np.abs(M_kminus1)).max(),(np.abs(M_kminus1)).argmax()

In [496]:
M_normalized = np.zeros(M_kminus1.shape) 

In [497]:
M_normalized[bMax] = 1/abs(M_kminus1[bMax])

In [516]:
un_k[k]

array([-0.02649328, -2.09729799, -9.77755794])

In [508]:
un_k[k] = (Lambda_k * M_normalized*np.sign(M_kminus1))

In [515]:
un_k[k] =  Lambda_k * (np.abs(M_kminus1)**(q-1))*np.sign(M_kminus1)/(M_qnorm**(q/p))

In [384]:
adaParams = adanet_init(numExamples,numInputNodes,cfg)

In [387]:
adaParams['lossStore'] = []
adaParams['numInputNodes'] = numInputNodes
adaParams['numOutputNodes'] = 1

In [ ]:
big_lambdas(n) = cfg.big_lambda/n_i^(1/q)

In [389]:
big_lambdas = np.zeros((adaParams['maxLayers'],1),float)

In [405]:
big_lambdas

array([[0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.71417785]])

In [570]:
numNodes[0]=5

In [571]:
numNodes[1]=10

In [588]:
np.zeros((numNodes.sum(dtype=np.int32),1))

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.]])

In [590]:
numNodes != 0

array([[ True],
       [ True],
       [False],
       [False],
       [False]])

In [397]:
q = adaParams['qnorm']

In [404]:
big_lambdas[n] = cfg['big_lambda']/n_i**(1/q)

In [591]:
type(un_k)

dict

In [592]:
un_k

{0: array([-0.02649328, -2.09729799, -9.77755794])}

In [594]:
un_kabs = {}
for k,v in un_k.items():
    un_kabs[k] = np.abs(v)

In [595]:
un_kabs

{0: array([0.02649328, 2.09729799, 9.77755794])}

In [596]:
len(un_kabs.keys())

1

In [603]:
dnMax = np.zeros((len(un_kabs.keys()),))                 #[dnMax,kBinNew] = max(cellfun(@max,dnAbs));
kBinNew = np.zeros((len(un_kabs.keys()),))               #[dnMax,kBinNew] = max(cellfun(@max,dnAbs));
for k,v in un_kabs.items():                               #[dnMax,kBinNew] = max(cellfun(@max,dnAbs));
    dnMax[k],kBinNew[k] = v.max(),v.argmax()

In [604]:
dnMax

array([9.77755794])

In [605]:
kBinNew

array([2.])

In [606]:
kBinNew.shape

(1,)

In [67]:
s = np.array([a1,a2,a3])

In [64]:
a1 = 50

In [65]:
a2 = 70

In [66]:
a3 = 10

In [68]:
s.argmax()

1

In [69]:
s

array([50, 70, 10])

In [656]:
t = 10

In [657]:
history['newN'].append(np.hstack((np.array([t]).reshape(1,-1),np.zeros((1,adaParams['maxLayers'])))))

In [625]:
adaParams['maxLayers']

5

In [643]:
np.hstack((np.array([t]).reshape(1,-1),np.zeros((1,adaParams['maxLayers']))))

array([[4., 0., 0., 0., 0., 0.]])

In [633]:
np.zeros((1,adaParams['maxLayers'])).shape

(1, 5)

In [640]:
np.array([t]).reshape(1,-1).shape

(1, 1)

In [661]:
history['newN']

[array([[4., 0., 0., 1., 0., 0.]]), array([[10.,  0.,  0.,  0.,  1.,  0.]])]

In [660]:
history['newN'][-1][0,kNew+1] = 1

In [666]:
s1 = 'bias'

In [670]:
if hasattr(s, 'lower'):
    print('hello')

In [674]:
if str(s1) == 'bias':
    print('hello')

hello


In [676]:
import numpy as np
import scipy.optimize as op

def Sigmoid(z):
    return 1/(1 + np.exp(-z));

def Gradient(theta,x,y):
    m , n = x.shape
    theta = theta.reshape((n,1));
    y = y.reshape((m,1))
    sigmoid_x_theta = Sigmoid(x.dot(theta));
    grad = ((x.T).dot(sigmoid_x_theta-y))/m;
    return grad.flatten();

def CostFunc(theta,x,y):
    m,n = x.shape; 
    theta = theta.reshape((n,1));
    y = y.reshape((m,1));
    term1 = np.log(Sigmoid(x.dot(theta)));
    term2 = np.log(1-Sigmoid(x.dot(theta)));
    term1 = term1.reshape((m,1))
    term2 = term2.reshape((m,1))
    term = y * term1 + (1 - y) * term2;
    J = -((np.sum(term))/m);
    return J;

# intialize X and y
X = np.array([[1,2,3],[1,3,4]]);
y = np.array([[1],[0]]);

m , n = X.shape;
initial_theta = np.zeros(n);
Result = op.minimize(fun = CostFunc, 
                                 x0 = initial_theta, 
                                 args = (X, y),
                                 method = 'TNC',
                                 jac = Gradient);
optimal_theta = Result.x;

In [677]:
optimal_theta

array([ 77.74469423, -25.66384335,  -4.04584001])

In [682]:
X

array([[1, 2, 3],
       [1, 3, 4]])

In [684]:
y

array([[1],
       [0]])

In [680]:
initial_theta

array([0., 0., 0.])

In [685]:
Result

     fun: 4.1386555453899555e-07
     jac: array([-2.14912216e-07, -3.30347818e-07, -5.45260034e-07])
 message: 'Local minimum reached (|pg| ~= 0)'
    nfev: 17
     nit: 7
  status: 0
 success: True
       x: array([ 77.74469423, -25.66384335,  -4.04584001])

In [1]:
import numpy as np

In [33]:
errLoss = np.zeros((50,1))

In [64]:
np.array([[4.7614433],[5],[5]]).shape[0]

3

In [65]:
errLoss[0:3,:] = np.array([[4.7614433],[5],[5]])

In [73]:
np.logspace(-4,-3,5)

array([0.0001    , 0.00017783, 0.00031623, 0.00056234, 0.001     ])

In [71]:
np.logical_and(np.logical_not(np.array([[True],[False],[False],[False],[False],[False]])), np.array([[True],[True],[True],[True],[True],[True]]))

array([[False],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True]])

In [19]:
Wk.shape

(1, 1)

In [31]:
data = np.append([[1, 2, 3], [4, 5, 6]], [[7, 8, 9]],axis=0)

In [36]:
u = np.array([[0.2,0.1,0.7]])

In [33]:
data = np.append(data,[[10, 20, 30]],axis=0)

In [35]:
data

array([[ 1,  2,  3],
       [ 4,  5,  6],
       [ 7,  8,  9],
       [10, 20, 30]])

In [49]:
u = u.reshape(-1,1)

In [50]:
u.shape

(3, 1)

In [51]:
hkm1 = np.dot(data,u)

In [54]:
hkm1.shape

(4, 1)

In [58]:
np.append(data,[[1],[2],[3],[4]], axis=1)

array([[ 1,  2,  3,  1],
       [ 4,  5,  6,  2],
       [ 7,  8,  9,  3],
       [10, 20, 30,  4]])

In [57]:
np.array([[1],[2],[3],[4]]).shape

(4, 1)

In [60]:
np.zeros((5,1),dtype=np.int)

array([[0],
       [0],
       [0],
       [0],
       [0]])

In [61]:
for i in range(1): 
    print(i)

0
